In [ ]:
# Import statements
import numpy as np
import matplotlib.pyplot as pt
import glob
import os

from ase.io import read
from pyiron import Project, ase_to_pyiron

from molmod.units import *
from molmod.constants import *

from collections import namedtuple
from dataclasses import dataclass # replaces namedtuple with mutable attributes
%matplotlib inline


### Functions

In [ ]:
# Static and dynamic jobtypes for GPXRD module
def static_PXRD_job(pr,name,structure,wavelength=1.54056,refpattern=None,cluster='victini',run_time=15*60):
    job = pr.create_job(pr.job_type.GPXRD, name, delete_existing_job=True)
    job.structure = structure
    job.input['jobtype'] = 'static'
    job.input['numpoints'] = 1001 
    job.input['wavelength'] = wavelength # in angstrom
    
    if refpattern is not None:
        job.set_reference_pattern(refpattern)
    
    # Submit this job to the queue
    job.server.queue = cluster
    job.server.cores = 1 
    job.server.run_time = run_time # in seconds

    job.run()
    #return job

def dynamic_PXRD_job(pr,name,xyz_traj,wavelength=1.54056,refpattern=None,start=0,stop=-1,num_frames=50,cluster='victini',run_time=15*60):
    job = pr.create_job(pr.job_type.GPXRD, name, delete_existing_job=True)
    job.load_trajectory(xyz_traj,start=start,stop=stop,num_frames=num_frames)
    
    job.input['numpoints'] = 1001
    job.input['wavelength'] = wavelength # in angstrom
    
    if refpattern is not None:
        job.set_reference_pattern(refpattern)
    
    # We dont want individual fhkl indices
    job.input['save_fhkl'] = False
    
    # Submit this job to the queue
    job.server.queue = cluster
    job.server.cores = 1
    job.server.run_time = run_time # in seconds

    job.run()
    #return job

In [ ]:
# Some code to convert CP2K trajectory data into Atoms objects
class CP2K_xyz(object):
    def __init__(self, fname, mode):
        from molmod.io.xyz import XYZFile
        import glob
        
        print(fname)
        # Assume fname is the directory containing the pos and cell file
        fname_pos = glob.glob(os.path.join(fname, '*.xyz'))[0] # assume it is only 1
        fname_cell = glob.glob(os.path.join(fname, '*.cell'))[0] # assume it is only 1
        
        
        #fname = os.path.splitext(fname)[0]
        #xyz = XYZFile(fname+'.xyz')
        xyz = XYZFile(fname_pos)
        
        self.geometries = xyz.geometries
        self.numbers = xyz.numbers
        self.symbols = xyz.symbols

        if mode=='npt':
            cell = np.loadtxt(fname_cell)[:, 2:-1]*angstrom
            cell = cell.reshape(-1, 3, 3)
            
            if cell.shape[0] != self.geometries.shape[0]:
                if cell.shape[0]>self.geometries.shape[0]:
                    # sub sample cell
                    print('Subsampling cell! Cell shape {}, Geometries shape {}'.format(cell.shape, self.geometries.shape))
                    step_size = int((cell.shape[0]-1)//(self.geometries.shape[0]-1))
                    print('Step size {}'.format(step_size))
                    cell = cell[::step_size]
                    print('New cell shape {}'.format(cell.shape))
                else:
                    raise NotImplementedError
            
        elif mode=='nvt':
            cell = np.loadtxt(fname_cell)[2:-1]*angstrom
            cell = cell.reshape(1, 3, 3)
            cell = np.repeat(cell,len(self.geometries),axis=0)
        else:
            raise ValueError("Mode not recognized")
        self.cell = cell
        
def read_xyz_traj(fname,start=0,stop=-1,num_frames=50,mode='npt'):
    from pyiron.atomistics.structure.atoms import Atoms
    
     # Load CP2K trajectory, seperate cell file
    cp2k_xyz = CP2K_xyz(fname,mode)    
    coords = cp2k_xyz.geometries/angstrom # in angstrom
    rvecs  = cp2k_xyz.cell/angstrom # in angstrom
    
    if stop < 0:
        stop = coords.shape[0] + stop + 1
    else:
        stop = min(stop, coords.shape[0])
    if start < 0:
        start = coords.shape[0] + start + 1

    if num_frames is None:
        num_frames = stop-start
        
    frames = np.linspace(start,stop,num_frames,dtype=int,endpoint=False)
        
    return [Atoms(positions=coords[i],numbers=cp2k_xyz.numbers,cell=rvecs[i], pbc=True) for i in frames]

### Execution

In [ ]:
# Create a project - acts as a directory to bundle all data
pr = Project('PXRD')

#### Static jobs

In [ ]:
# Subprojects allow for easy subdivision
pr_static = pr.create_group('static') # equivalent to pr_static = Project('PXRD/static')

In [ ]:
# Load input structures (typically cifs)
cifs = glob.glob('./input_files/PXRD/*.cif')
structures = { 
    c.split('/')[-1].split('.')[0]  : ase_to_pyiron(read(c)) for c in cifs
}

# ase handles the conversion from most structure types

In [ ]:
for k,v in structures.items():
    static_PXRD_job(pr_static,k,v,wavelength=1.54056,rad_type=rad_type,refpattern=None,run_time=15*60)

#### Dynamic jobs

In [ ]:
# Subprojects allow for easy subdivision
pr_dynamic = pr.create_group('dynamic') # equivalent to pr_dynamic = Project('PXRD/dynamic')

In [ ]:
# this may take a while depending on the size of this file
xyz_traj = read_xyz_traj('./input_files/PXRD/trajectory.xyz', start=1000,stop=-1,num_frames=10)

In [ ]:
# Calculate dynamically averaged PXRD
dynamic_PXRD_job(pr_dynamic,'job_name',xyz_traj,refpattern=None,start=0,stop=-1,num_frames=None) # take all frames